In [19]:
import pandas as pd
import numpy as np
import os
import glob
import os
import subprocess
import pandas as pd
import numpy as np
from scipy.stats import iqr
from statsmodels.formula.api import ols
from scipy.stats import zscore
from nipype.interfaces.fsl import ImageMaths
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
# Create bar plot
plt.style.use('seaborn-v0_8-poster')
#plt.style.use('seaborn-v0_8')
fontsize = 11
plt.rcParams.update({
    "font.size": fontsize,          # Global font size
    "axes.labelsize": fontsize,     # X and Y axis labels
    "axes.titlesize": fontsize,     # Title (if used)
    "xtick.labelsize": fontsize,    # X-axis tick labels
    "ytick.labelsize": fontsize,    # Y-axis tick labels
    "legend.fontsize": fontsize,    # Legend font size
})

In [20]:
project_dir = '/Users/xiaoqianxiao/projects/OCD'
data_dir = os.path.join(project_dir, 'results/NW_group_network_network')
result_dir = os.path.join(project_dir, 'results/visualization/NW_group_network_network')
if not os.path.exists(result_dir):
    os.makedirs(result_dir, exist_ok=True)

In [31]:
group_diff_file = os.path.join(data_dir, 'group_diff_baseline_power_2011_network_fc.csv')
df_group_diff = pd.read_csv(group_diff_file)
df_group_diff['network1'] = df_group_diff['Feature'].str.split('_').str[0]
df_group_diff['network2'] = df_group_diff['Feature'].str.split('_').str[-2]
df_group_diff.sort_values(by='p_value', ascending=True).head(20)

,Feature,t_statistic,p_value,OCD_mean,HC_mean,OCD_n,HC_n,p_value_fdr,network1,network2
28,Cerebellar_Sensory_Somatomotor_Mouth_fc,2.078920,0.044728,0.199593,0.142780,46,46,0.981125,Cerebellar,Mouth
102,Cerebellar_Cerebellar_fc,1.841498,0.072148,0.323174,0.268699,46,46,0.981125,Cerebellar,Cerebellar
103,Cerebellar_Dorsal_attention_fc,1.779875,0.081120,0.168711,0.126686,46,46,0.981125,Cerebellar,attention
6,Memory_retrieval_Uncertain_fc,1.698304,0.095722,0.119767,0.088901,46,46,0.981125,Memory,Uncertain
64,Default_mode_Memory_retrieval_fc,1.646496,0.106225,0.192945,0.157061,46,46,0.981125,Default,retrieval
96,Dorsal_attention_Salience_fc,1.506933,0.139927,0.153600,0.119622,46,46,0.981125,Dorsal,Salience
89,Visual_Visual_fc,-1.478028,0.145656,0.274435,0.310829,46,46,0.981125,Visual,Visual
84,Cerebellar_Visual_fc,1.465660,0.150749,0.182745,0.147971,46,46,0.981125,Cerebellar,Visual
76,Memory_retrieval_Visual_fc,1.425610,0.160427,0.117844,0.082488,46,46,0.981125,Memory,Visual
35,Sensory_Somatomotor_Mouth_Sensory_Somatomotor_...,-1.328484,0.190226,0.397753,0.450028,46,46,0.981125,Sensory,Mouth


In [22]:
df_group_diff.loc[
    (df_group_diff['p_value'] <= 0.05)
]

,Feature,t_statistic,p_value,OCD_mean,HC_mean,OCD_n,HC_n,p_value_fdr,network1,network2
28,Cerebellar_Sensory_Somatomotor_Mouth_fc,2.07892,0.044728,0.199593,0.14278,46,46,0.981125,Cerebellar,Sensory


In [34]:
df_group_diff.loc[
    (df_group_diff['network1'].isin(['Default', 'Dorsal', 'attention', 'mode'])) &
    (df_group_diff['network2'].isin(['Default', 'Dorsal', 'attention', 'mode'])) &
    (df_group_diff['p_value'] <= 0.05)]

,Feature,t_statistic,p_value,OCD_mean,HC_mean,OCD_n,HC_n,p_value_fdr,network1,network2


In [9]:
baselineFC_vs_deltaYBOCS_file = os.path.join(data_dir, 'baselineFC_vs_deltaYBOCS_power_2011_network_fc.csv')
df_baselineFC_vs_deltaYBOCS = pd.read_csv(baselineFC_vs_deltaYBOCS_file)
df_baselineFC_vs_deltaYBOCS['network1'] = df_baselineFC_vs_deltaYBOCS['Feature'].str.split('_').str[0]
df_baselineFC_vs_deltaYBOCS['network2'] = df_baselineFC_vs_deltaYBOCS['Feature'].str.split('_').str[1]
df_baselineFC_vs_deltaYBOCS.sort_values(by='fc_p_value', ascending=True).head(20)

,Feature,fc_effect,fc_p_value,r_value,n,condition_effects,condition_p_values,model_r_squared,model_adj_r_squared,fc_p_value_fdr,network1,network2
41,Cingulo-opercular_Task_Control_Cingulo-opercul...,-0.003720,0.125444,-0.239854,44,{'Wait': -0.02406380846868318},{'Wait': 0.38018900837019},0.075279,0.030171,0.993668,Cingulo-opercular,Task
104,Dorsal_attention_Dorsal_attention_fc,-0.003610,0.156453,-0.218302,44,{'Wait': 0.00844274109268791},{'Wait': 0.7687241213488512},0.049687,0.003330,0.993668,Dorsal,attention
83,Ventral_attention_Visual_fc,0.002549,0.181288,0.206260,44,{'Wait': -0.005855245273149923},{'Wait': 0.7855553283070618},0.044291,-0.002328,0.993668,Ventral,attention
46,Cingulo-opercular_Task_Control_Salience_fc,-0.002747,0.186996,-0.208070,44,{'Wait': -0.022743398570362144},{'Wait': 0.33567219238157764},0.064938,0.019325,0.993668,Cingulo-opercular,Task
90,Cerebellar_Fronto-parietal_Task_Control_fc,-0.002648,0.191665,-0.204836,44,{'Wait': -0.04209409179869179},{'Wait': 0.07147892782772625},0.115795,0.072663,0.993668,Cerebellar,Fronto-parietal
67,Default_mode_Ventral_attention_fc,0.002444,0.197067,0.194190,44,{'Wait': -0.019057845647346652},{'Wait': 0.3751507248521575},0.056216,0.010178,0.993668,Default,mode
40,Cerebellar_Cingulo-opercular_Task_Control_fc,-0.003027,0.215524,-0.195615,44,{'Wait': -0.04323467523690513},{'Wait': 0.12291229905407923},0.093142,0.048905,0.993668,Cerebellar,Cingulo-opercular
33,Memory_retrieval_Sensory_Somatomotor_Mouth_fc,0.003878,0.257220,0.169317,44,{'Wait': -0.03877713389254324},{'Wait': 0.3196370450847333},0.052132,0.005894,0.993668,Memory,retrieval
30,Default_mode_Sensory_Somatomotor_Mouth_fc,0.002942,0.258965,0.164054,44,{'Wait': -0.04313100947248562},{'Wait': 0.148806099534155},0.075723,0.030637,0.993668,Default,mode
101,Subcortical_Subcortical_fc,0.002760,0.299584,0.161055,44,{'Wait': -0.0050438457353059254},{'Wait': 0.8671285894928665},0.026612,-0.020871,0.993668,Subcortical,Subcortical


In [42]:
df_baselineFC_vs_deltaYBOCS.loc[
    (df_baselineFC_vs_deltaYBOCS['fc_p_value'] <= 0.05)
]

,Feature,fc_effect,fc_p_value,r_value,n,condition_effects,condition_p_values,model_r_squared,model_adj_r_squared,fc_p_value_fdr,network1,network2


In [35]:
df_baselineFC_vs_deltaYBOCS.loc[
    (df_baselineFC_vs_deltaYBOCS['network1'].isin(['Default', 'Dorsal', 'attention', 'mode'])) &
    (df_baselineFC_vs_deltaYBOCS['network2'].isin(['Default', 'Dorsal', 'attention', 'mode'])) &
    (df_baselineFC_vs_deltaYBOCS['fc_p_value'] <= 0.05)
]

,Feature,fc_effect,fc_p_value,r_value,n,condition_effects,condition_p_values,model_r_squared,model_adj_r_squared,fc_p_value_fdr,network1,network2


In [36]:
deltaFC_vs_deltaYBOCS_file = os.path.join(data_dir, 'deltaFC_vs_deltaYBOCS_power_2011_network_fc.csv')
df_deltaFC_vs_deltaYBOCS = pd.read_csv(deltaFC_vs_deltaYBOCS_file)
df_deltaFC_vs_deltaYBOCS['network1'] = df_deltaFC_vs_deltaYBOCS['Feature'].str.split('_').str[0]
df_deltaFC_vs_deltaYBOCS['network2'] = df_deltaFC_vs_deltaYBOCS['Feature'].str.split('_').str[-2]
df_deltaFC_vs_deltaYBOCS.sort_values(by='fc_p_value', ascending=True).head(20)

,Feature,fc_effect,fc_p_value,r_value,n,condition_effects,condition_p_values,model_r_squared,model_adj_r_squared,fc_p_value_fdr,network1,network2
98,Subcortical_Visual_fc,-0.005426,0.023469,-0.349091,43,{'Wait': -0.02408054533129324},{'Wait': 0.36932404374284533},0.139598,0.096578,0.984686,Subcortical,Visual
91,Sensory_Somatomotor_Mouth_Subcortical_fc,-0.005416,0.032263,-0.329338,43,{'Wait': 0.010313453288719812},{'Wait': 0.7156170533619055},0.111454,0.067027,0.984686,Sensory,Subcortical
102,Ventral_attention_Ventral_attention_fc,-0.005604,0.060450,-0.293215,43,{'Wait': -0.018425261076697255},{'Wait': 0.5842284189445034},0.092878,0.047522,0.984686,Ventral,attention
97,Subcortical_Ventral_attention_fc,-0.003748,0.067104,-0.285315,43,{'Wait': -0.02551551412363807},{'Wait': 0.27244970878418706},0.108987,0.064437,0.984686,Subcortical,attention
95,Subcortical_Subcortical_fc,-0.004484,0.079966,-0.272579,43,{'Wait': -0.0388046423642931},{'Wait': 0.1845641970508387},0.114646,0.070379,0.984686,Subcortical,Subcortical
72,Memory_retrieval_Sensory_Somatomotor_Mouth_fc,-0.006246,0.092705,-0.261765,43,{'Wait': 0.010308609008049969},{'Wait': 0.8062537875576888},0.069938,0.023435,0.984686,Memory,Mouth
10,Auditory_Subcortical_fc,-0.003890,0.101878,-0.256631,43,{'Wait': -0.027765851528307003},{'Wait': 0.305614600894963},0.090351,0.044868,0.984686,Auditory,Subcortical
12,Auditory_Ventral_attention_fc,-0.004347,0.113961,-0.248814,43,{'Wait': -0.01773114263046677},{'Wait': 0.570212310990502},0.069532,0.023008,0.984686,Auditory,attention
23,Cerebellar_Subcortical_fc,-0.003703,0.152261,-0.225658,43,{'Wait': -0.006835646438743185},{'Wait': 0.816214868168208},0.052219,0.004829,0.984686,Cerebellar,Subcortical
45,Default_mode_Sensory_Somatomotor_Mouth_fc,-0.004167,0.153392,-0.215591,43,{'Wait': 0.04119979485573247},{'Wait': 0.2186559957856627},0.082312,0.036427,0.984686,Default,Mouth


In [43]:
df_deltaFC_vs_deltaYBOCS.loc[
    (df_deltaFC_vs_deltaYBOCS['fc_p_value'] <= 0.05)
]

,Feature,fc_effect,fc_p_value,r_value,n,condition_effects,condition_p_values,model_r_squared,model_adj_r_squared,fc_p_value_fdr,network1,network2
91,Sensory_Somatomotor_Mouth_Subcortical_fc,-0.005416,0.032263,-0.329338,43,{'Wait': 0.010313453288719812},{'Wait': 0.7156170533619055},0.111454,0.067027,0.984686,Sensory,Subcortical
98,Subcortical_Visual_fc,-0.005426,0.023469,-0.349091,43,{'Wait': -0.02408054533129324},{'Wait': 0.36932404374284533},0.139598,0.096578,0.984686,Subcortical,Visual


In [44]:
df_deltaFC_vs_deltaYBOCS.loc[
    (df_deltaFC_vs_deltaYBOCS['network1'].isin(['Default', 'Dorsal', 'attention', 'mode'])) &
    (df_deltaFC_vs_deltaYBOCS['network2'].isin(['Default', 'Dorsal', 'attention', 'mode'])) &
    (df_deltaFC_vs_deltaYBOCS['fc_p_value'] <= 0.05)
]

,Feature,fc_effect,fc_p_value,r_value,n,condition_effects,condition_p_values,model_r_squared,model_adj_r_squared,fc_p_value_fdr,network1,network2


In [38]:
condition_baseline_FC_file = os.path.join(data_dir, 'condition_baseline_FC_power_2011_network_fc.csv')
condition_baseline_FC = pd.read_csv(condition_baseline_FC_file)
condition_baseline_FC['network1'] = condition_baseline_FC['Feature'].str.split('_').str[0]
condition_baseline_FC['network2'] = condition_baseline_FC['Feature'].str.split('_').str[-2]
condition_baseline_FC.sort_values(by='p_value', ascending=True).head(20)

,Feature,condition1,condition2,t_statistic,p_value,condition1_mean,condition2_mean,condition1_n,condition2_n,effect_size,p_value_fdr,network1,network2
26,Cerebellar_Visual_fc,Treat,Wait,-2.821025,0.007202,0.211413,0.151471,24,22,-0.824537,0.429931,Cerebellar,Visual
24,Cerebellar_Uncertain_fc,Treat,Wait,-2.774248,0.008189,0.145567,0.097147,24,22,-0.808871,0.429931,Cerebellar,Uncertain
14,Cerebellar_Cerebellar_fc,Treat,Wait,-2.508604,0.015911,0.359986,0.283014,24,22,-0.734659,0.556882,Cerebellar,Cerebellar
18,Cerebellar_Fronto-parietal_Task_Control_fc,Treat,Wait,-2.018571,0.049652,0.116381,0.072096,24,22,-0.593445,0.821351,Cerebellar,Control
36,Cingulo-opercular_Task_Control_Uncertain_fc,Treat,Wait,-1.980387,0.054097,0.108907,0.071314,24,22,-0.578156,0.821351,Cingulo-opercular,Uncertain
92,Sensory_Somatomotor_Mouth_Uncertain_fc,Treat,Wait,-1.796517,0.080576,0.113941,0.079304,24,22,-0.518334,0.821351,Sensory,Uncertain
17,Cerebellar_Dorsal_attention_fc,Treat,Wait,-1.746175,0.088345,0.191622,0.143718,24,22,-0.506935,0.821351,Cerebellar,attention
30,Cingulo-opercular_Task_Control_Fronto-parietal...,Treat,Wait,-1.682549,0.099561,0.121162,0.079096,24,22,-0.495663,0.821351,Cingulo-opercular,Control
1,Auditory_Cerebellar_fc,Treat,Wait,-1.671808,0.102201,0.192042,0.151018,24,22,-0.485621,0.821351,Auditory,Cerebellar
11,Auditory_Uncertain_fc,Treat,Wait,-1.671821,0.102675,0.113483,0.084360,24,22,-0.483490,0.821351,Auditory,Uncertain


In [39]:
condition_followup_FC_file = os.path.join(data_dir, 'condition_followup_FC_power_2011_network_fc.csv')
condition_followup_FC = pd.read_csv(condition_followup_FC_file)
condition_followup_FC['network1'] = condition_followup_FC['Feature'].str.split('_').str[0]
condition_followup_FC['network2'] = condition_followup_FC['Feature'].str.split('_').str[-2]
condition_followup_FC.sort_values(by='p_value', ascending=True).head(20)

,Feature,condition1,condition2,t_statistic,p_value,condition1_mean,condition2_mean,condition1_n,condition2_n,effect_size,p_value_fdr,network1,network2
13,Auditory_Visual_fc,Treat,Wait,-2.610189,0.012654,0.203212,0.139920,21,22,-0.798103,0.982651,Auditory,Visual
26,Cerebellar_Visual_fc,Treat,Wait,-2.268361,0.028877,0.209831,0.153281,21,22,-0.694685,0.982651,Cerebellar,Visual
73,Memory_retrieval_Subcortical_fc,Treat,Wait,-1.813861,0.077025,0.132883,0.084078,21,22,-0.552487,0.982651,Memory,Subcortical
17,Cerebellar_Dorsal_attention_fc,Treat,Wait,-1.806535,0.078199,0.188560,0.135330,21,22,-0.551203,0.982651,Cerebellar,attention
69,Memory_retrieval_Memory_retrieval_fc,Treat,Wait,-1.801307,0.079068,0.386437,0.329292,21,22,-0.547789,0.982651,Memory,retrieval
95,Subcortical_Subcortical_fc,Treat,Wait,-1.632612,0.110218,0.230454,0.187091,21,22,-0.497936,0.982651,Subcortical,Subcortical
38,Cingulo-opercular_Task_Control_Visual_fc,Treat,Wait,-1.630583,0.110641,0.158495,0.116031,21,22,-0.497123,0.982651,Cingulo-opercular,Visual
46,Default_mode_Subcortical_fc,Treat,Wait,-1.604344,0.116701,0.120874,0.081357,21,22,-0.491532,0.982651,Default,Subcortical
57,Dorsal_attention_Uncertain_fc,Treat,Wait,-1.523026,0.135443,0.118260,0.086063,21,22,-0.464594,0.982651,Dorsal,Uncertain
89,Sensory_Somatomotor_Hand_Visual_fc,Treat,Wait,-1.502620,0.140611,0.219060,0.179819,21,22,-0.457499,0.982651,Sensory,Visual


In [40]:
condition_FC_change_file = os.path.join(data_dir, 'condition_FC_change_power_2011_network_fc.csv')
condition_FC_change = pd.read_csv(condition_FC_change_file)
condition_FC_change['network1'] = condition_FC_change['Feature'].str.split('_').str[0]
condition_FC_change['network2'] = condition_FC_change['Feature'].str.split('_').str[-2]
condition_FC_change.sort_values(by='p_value', ascending=True).head(20)

,Feature,condition1,condition2,t_statistic,p_value,condition1_mean,condition2_mean,condition1_n,condition2_n,effect_size,p_value_fdr,network1,network2
77,Salience_Salience_fc,Treat,Wait,2.368064,0.022763,-0.028379,0.031676,21,22,0.723868,0.960953,Salience,Salience
30,Cingulo-opercular_Task_Control_Fronto-parietal...,Treat,Wait,1.965963,0.056119,-0.038975,0.014365,21,22,0.599858,0.960953,Cingulo-opercular,Control
62,Fronto-parietal_Task_Control_Salience_fc,Treat,Wait,1.930851,0.060635,-0.035629,0.009638,21,22,0.586052,0.960953,Fronto-parietal,Salience
92,Sensory_Somatomotor_Mouth_Uncertain_fc,Treat,Wait,1.650671,0.106764,-0.024785,0.011397,21,22,0.505473,0.960953,Sensory,Uncertain
33,Cingulo-opercular_Task_Control_Sensory_Somatom...,Treat,Wait,1.647299,0.107233,-0.020407,0.034087,21,22,0.503304,0.960953,Cingulo-opercular,Hand
29,Cingulo-opercular_Task_Control_Dorsal_attentio...,Treat,Wait,1.620915,0.113443,-0.013943,0.029578,21,22,0.497645,0.960953,Cingulo-opercular,attention
32,Cingulo-opercular_Task_Control_Salience_fc,Treat,Wait,1.554239,0.128192,-0.000069,0.039967,21,22,0.476153,0.960953,Cingulo-opercular,Salience
64,Fronto-parietal_Task_Control_Sensory_Somatomot...,Treat,Wait,1.516854,0.138074,-0.035974,0.009104,21,22,0.458125,0.960953,Fronto-parietal,Mouth
73,Memory_retrieval_Subcortical_fc,Treat,Wait,-1.495415,0.142532,0.010238,-0.029060,21,22,-0.454689,0.960953,Memory,Subcortical
84,Sensory_Somatomotor_Hand_Sensory_Somatomotor_H...,Treat,Wait,1.478238,0.147031,-0.016307,0.035791,21,22,0.451299,0.960953,Sensory,Hand
